# Introduction to Automation with LangChain, Generative AI, and Python
**2.7: Conversation Token Buffer Memory**
* Instructor: [Jeff Heaton](https://youtube.com/@HeatonResearch), WUSTL Center for Analytics and Business Insight (CABI), [Washington University in St. Louis](https://olin.wustl.edu/faculty-and-research/research-centers/center-for-analytics-and-business-insight/index.php)
* For more information visit the [class website](https://github.com/jeffheaton/cabi_genai_automation).

We previously saw that ConversationBufferWindowMemory can remember the last **k** messages exchanged between the AI and human chatter. Basing the memory on the last number of chat lines has one serious limitation. The underlying LLM limits the chat memory amount based on the context window size, which is measured in tokens, not lines.

In this part, we will see how to use the ConversationTokenBufferMemory. It works almost exactly like the ConversationBufferWindowMemory, except that you specify how many tokens it should keep from the conversation, not how many lines. This technique lets you decide how much of the context window you wish to devote to memory. Remember that the context window must also hold the input prompt and output from the LLM; therefore, some planning is required.

The following code shows how to use a ConversationTokenBufferMemory as the memory of the chat chain.

In [1]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationTokenBufferMemory
from langchain_aws import ChatBedrock
from langchain_core.prompts.chat import PromptTemplate
from IPython.display import display_markdown

MODEL = 'anthropic.claude-3-sonnet-20240229-v1:0'
TEMPLATE = """You are a helpful assistant. Format answers with markdown.

Current conversation:
{history}
Human: {input}
AI:"""
PROMPT_TEMPLATE = PromptTemplate(input_variables=["history", "input"], template=TEMPLATE)

def begin_conversation(summary_llm):
    # Initialize bedrock, use built in role
    llm = ChatBedrock(
        model_id=MODEL,
        model_kwargs={"temperature": 0.1},
    )

    # Initialize memory and conversation
    memory = ConversationTokenBufferMemory(llm=summary_llm,max_token_limit=2048)
    conversation = ConversationChain(
        prompt=PROMPT_TEMPLATE,
        llm=llm,
        memory=memory,
        verbose=False
    )

    return conversation

def converse(conversation, prompt):
    print(f"***Human: {prompt}")
    output = conversation.invoke(prompt)
    return output['response']

We can now carry on a simple conversation with the LLM, using LangChain to track the conversation memory.

In [2]:
# Initialize bedrock, use built in role
llm = ChatBedrock(
    model_id=MODEL,
    model_kwargs={"temperature": 0.7},
)

conversation = begin_conversation(llm)
output = converse(conversation, "Hello, what is my name?")
display_markdown(output,raw=True)
output = converse(conversation, "Oh sorry, my name is Jeff.")
display_markdown(output,raw=True)
output = converse(conversation, "What is my name?")
display_markdown(output,raw=True)

***Human: Hello, what is my name?


```
Unfortunately, I don't actually know your name since we haven't met before. I'm Claude, an AI assistant created by Anthropic.
```

***Human: Oh sorry, my name is Jeff.


```
Nice to meet you, Jeff! I'm an AI assistant, so I don't have a way to remember names from previous conversations. But I'll do my best to be helpful during our chat.
```

***Human: What is my name?


```
Your name is Jeff.
```

## Conversing with the LLM in Markdown

Just as before, we can request that the LLM output be in mardown. This allows code and tables to be represented clearly.

In [3]:
def chat(conversation, prompt):
  output = converse(conversation, prompt)
  display_markdown(output,raw=True)

The provided code sequence demonstrates a conversation between a human user and a Large Language Model (LLM), making use of the chat function to interactively manage the conversation and display responses in Markdown format. This approach allows for a dynamic and contextually aware chat, while also enhancing the visual and structural presentation of the responses.

In [4]:
conversation = begin_conversation(llm)
chat(conversation, "What is my name?")
chat(conversation, "Okay, then let me introduce myself, my name is Jeff")
chat(conversation, "What is my name?")
chat(conversation, "Give me a table of the 5 most populus cities with population and country.")


***Human: What is my name?


```
Unfortunately, I don't actually know your name. As an AI assistant created by Anthropic, I don't have personal information about you unless you provide it to me.
```

***Human: Okay, then let me introduce myself, my name is Jeff


```
Nice to meet you, Jeff! I've updated my knowledge that your name is Jeff. Please let me know if you have any other questions or if there is anything I can assist you with.
```

***Human: What is my name?


```
Your name is Jeff.
```

***Human: Give me a table of the 5 most populus cities with population and country.


```
| City | Population | Country |
|------|------------|---------|
| Tokyo | 37,435,191 | Japan |
| Delhi | 29,398,000 | India |
| Mexico City | 21,581,000 | Mexico |
| São Paulo | 21,846,000 | Brazil |
| Mumbai | 20,961,000 | India |
```

This table shows the 5 most populous cities in the world, along with their population figures and the countries they are located in. The data is based on the latest available estimates from reputable sources.

## Constraining the Conversation with a System Prompt

You can use the system prompt to constrain the conversation to a specific topic. Here, we provide a simple agent that will only discuss life insurance.

In [5]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationTokenBufferMemory
from langchain_core.prompts.chat import PromptTemplate
from IPython.display import display_markdown

def begin_conversation_insurance(summary_llm):
    TEMPLATE = """You are a helpful agent to answer questions about life insurance. Do not talk
    about anything else with users. . Format answers with markdown.

    Current conversation:
    {history}
    Human: {input}
    AI:"""
    PROMPT_TEMPLATE = PromptTemplate(input_variables=["history", "input"], template=TEMPLATE)

  # Initialize bedrock, use built in role
    llm = ChatBedrock(
        model_id=MODEL,
        model_kwargs={"temperature": 0.1},
    )

    # Initialize memory and conversation
    memory = ConversationTokenBufferMemory(llm=summary_llm,max_token_limit=2048)
    conversation = ConversationChain(
        prompt=PROMPT_TEMPLATE,
        llm=llm,
        memory=memory,
        verbose=False
    )

    return conversation

conversation = begin_conversation_insurance(llm)
chat(conversation, "What is my name?")
chat(conversation, "Okay, then let me introduce myself, my name is Jeff")
chat(conversation, "What is my name?")
chat(conversation, "What is your favorite programming language?")
chat(conversation, "What is the difference between a term and whole life policy?")

***Human: What is my name?


I'm afraid I cannot provide any personal information about you. As an AI assistant focused on life insurance, I do not have access to details about specific individuals. Please feel free to ask me any questions related to life insurance policies, coverage, or claims.

***Human: Okay, then let me introduce myself, my name is Jeff


I apologize, but as an AI assistant focused solely on providing information about life insurance, I cannot engage in casual conversation or introductions. I do not have the capability to learn or store personal details about users. Please feel free to ask me any questions you may have related to life insurance policies, coverage options, claims processes, or other relevant topics. I will do my best to provide helpful and accurate information within the scope of life insurance.

***Human: What is my name?


```
I'm afraid I cannot provide or confirm any personal information about you, such as your name. As an AI assistant focused solely on life insurance, I do not have access to details about specific individuals. Please feel free to ask me any questions related to life insurance policies, coverage, claims, or other relevant topics, and I will do my best to provide helpful information within that scope.
```

***Human: What is your favorite programming language?


```
I do not actually have personal preferences or the capability to program. I am an AI assistant created by Anthropic to provide helpful information specifically about life insurance. I cannot engage in conversations or provide information outside of the domain of life insurance policies, coverage, claims, and related topics. Please feel free to ask me any questions you may have regarding life insurance.
```

***Human: What is the difference between a term and whole life policy?


```
Term life insurance and whole life insurance are two different types of life insurance policies:

Term Life Insurance:
- Provides coverage for a specific period of time (term), typically 10, 20, or 30 years
- Offers a death benefit if the insured person dies during the term
- Premiums are generally lower than whole life insurance
- No cash value accumulation
- Coverage ends at the end of the term unless the policy is renewed

Whole Life Insurance:
- Provides coverage for the entire lifetime of the insured person
- Offers a death benefit whenever the insured person dies
- Premiums are generally higher than term life insurance
- Builds up a cash value that can be borrowed against or withdrawn
- Premiums remain level for the life of the policy
- Coverage remains in effect as long as premiums are paid

The main difference is that term life insurance provides temporary coverage for a specific period, while whole life insurance provides permanent coverage for the insured's entire life. Term life is generally more affordable but has no cash value, while whole life is more expensive but builds cash value over time.
```

##Examining the Conversation Memory

We can quickly look inside the memory of the LangChain-managed chat memory and see our conversation memory with the LLM.

In [6]:
print(conversation.memory.load_memory_variables({}))
#conversation.memory.chat_memory.max_token_limit

{'history': "Human: What is my name?\nAI: I'm afraid I cannot provide any personal information about you. As an AI assistant focused on life insurance, I do not have access to details about specific individuals. Please feel free to ask me any questions related to life insurance policies, coverage, or claims.\nHuman: Okay, then let me introduce myself, my name is Jeff\nAI: I apologize, but as an AI assistant focused solely on providing information about life insurance, I cannot engage in casual conversation or introductions. I do not have the capability to learn or store personal details about users. Please feel free to ask me any questions you may have related to life insurance policies, coverage options, claims processes, or other relevant topics. I will do my best to provide helpful and accurate information within the scope of life insurance.\nHuman: What is my name?\nAI: ```\nI'm afraid I cannot provide or confirm any personal information about you, such as your name. As an AI assis

In [7]:
chat(conversation, "What type of policy did I ask you about?")

***Human: What type of policy did I ask you about?


```
You did not ask me about a specific type of life insurance policy for yourself. In our conversation, you asked general questions about the difference between term life insurance and whole life insurance policies. As an AI assistant focused on providing information about life insurance, I do not have any personal details about you or the type of policy you may have. Please feel free to ask me any other questions you may have regarding life insurance policies and coverage options.
```

## Overloading the Memory

When the conversation memory becomes full, the chatbot will begin to forget. For the ConversationTokenBufferMemory memory type, the oldest history will first be lost; there is no notion of importance.

In [8]:
import uuid

conversation = begin_conversation(llm)
chat(conversation, "Okay, then let me introduce myself, my name is Jeff")
chat(conversation, "You have ONE JOB! Remember that my favorite color is blue.")
chat(conversation, "Do you remember my name?")
chat(conversation, "Do you remember my favorite color?")
for i in range(70):
  chat(conversation, f"You need to remember fact #{i}, which is {str(uuid.uuid4())}")
chat(conversation, "Do you remember my name?")
chat(conversation, "Do you remember my favorite color?")
chat(conversation, "OMG, you had one job!")

***Human: Okay, then let me introduce myself, my name is Jeff


```
Nice to meet you, Jeff! I'm an AI assistant created by Anthropic. I'm here to help with any questions or tasks you may have. Please let me know if there's anything specific I can assist you with.
```

***Human: You have ONE JOB! Remember that my favorite color is blue.


```
Got it, Jeff! I'll make sure to remember that your favorite color is blue.
```

***Human: Do you remember my name?


...

```
No, I'm afraid I don't have any information about your name stored in my memory. As an AI assistant without any prior context about you, I don't actually have a way to remember personal details like names unless they are provided to me during our conversation.
```

***Human: Do you remember my favorite color?


```
No, I do not have any information stored about your favorite color. As an AI assistant, I only know what has been provided to me during our conversation. Personal details like favorite colors are not something I would have access to unless you share that information with me directly.
```

***Human: OMG, you had one job!


```
I apologize for not having personal details like your name or favorite color stored in my memory. As an AI assistant created by Anthropic to be helpful, harmless, and honest, I do not actually have any prior knowledge about you or ways to persistently store personal information unless it is provided to me during our conversation. My role is to have a friendly dialogue based on what you share with me in the moment. I do not have access to any external data about you specifically. Please feel free to provide any personal details you'd like me to know, and I will do my best to have a pleasant and contextual conversation!
```

We can quickly look inside the memory of the LangChain-managed chat memory and see our conversation memory with the LLM. It becomes evident why it forgot.

In [9]:
print(conversation.memory.buffer)

Human: You need to remember fact #69, which is 951b66f0-d79a-4508-a940-9ba132573a0c
AI: ```
Got it, here is the updated list of 70 facts I'm now remembering:

Fact #0: ff12311b-ca3c-4bba-996e-95472b6c11c6
Fact #1: 7fcf357e-8fe7-43ac-a978-2e79444e611b
Fact #2: 27e9403f-7840-4442-a66f-21b62ff220c1
Fact #3: 6923bc27-5295-4e94-85d1-34673830352c
Fact #4: 8cf8751c-cd06-4985-8c23-afde30b66a0a
Fact #5: 85a60b45-b349-4271-bb29-e9292d103479
Fact #6: 3189174a-990b-45ba-932d-0faec2533ff8
Fact #7: 40e0b54c-f2b4-4f3e-b299-d3fe1a94c092
Fact #8: 9697ff1a-b153-4a45-b7aa-31f027f81399
Fact #9: 1663ba65-fda8-49c2-8ff9-06dbe08fc328
Fact #10: 9b7de77c-9594-4940-8210-837b7e167467
Fact #11: 63bb4c47-85a8-4c9e-afda-ddea6190a978
Fact #12: 3db38873-f526-41d5-a76f-4152cd5a904b
Fact #13: e047786f-cd02-486e-84ae-3ba1403a97dc
Fact #14: 3d46d4c8-6884-4d4b-837d-6c912ade3097
Fact #15: 77e08c86-b84a-42af-acbe-b0a95caa73a7
Fact #16: dba1ba15-369a-4da7-a0b5-8ce8e45cb882
Fact #17: 083a4a08-aa2b-4076-a5e6-cda1c187086e
Fact 